In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
data = pd.read_pickle('data/merged.pickle')

In [3]:
clean = data
data.head()

,session_id,time,user_id,product_id,event,discount,purchase_id,made_purchase,known_product,product,...,duration,syntetic_duration,agg_time_spent,weekday,hour,weekday_sin,weekday_cos,hour_sin,hour_cos,ends_with_purchase
0,100001.0,2020-01-17 16:08:57,102.0,1001.0,view_product,0,NaN,False,True,telefon siemens gigaset da310,...,00:02:43.251191,True,00:08:06.502383,4,16,-0.433884,-0.900969,-0.866025,-0.500000,False
1,100002.0,2020-01-07 05:10:41,102.0,1277.0,view_product,20,NaN,False,True,apple ipad mini 64gb 4g,...,00:03:45,False,00:05:23.251191,1,5,0.781831,0.623490,0.965926,0.258819,False
2,100002.0,2020-01-07 05:14:26,102.0,1276.0,view_product,20,NaN,False,True,apple ipad mini 64gb,...,00:02:43.251191,True,00:03:45,1,5,0.781831,0.623490,0.965926,0.258819,False
3,100003.0,2020-01-16 09:37:42,102.0,1276.0,view_product,0,NaN,False,True,apple ipad mini 64gb,...,00:00:40,False,00:06:28.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False
4,100003.0,2020-01-16 09:38:22,102.0,1277.0,view_product,0,NaN,False,True,apple ipad mini 64gb 4g,...,00:02:43.251191,True,00:09:08.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False


In [4]:
clean.columns

Index(['session_id', 'time', 'user_id', 'product_id', 'event', 'discount',
       'purchase_id', 'made_purchase', 'known_product', 'product', 'price',
       'cat_0', 'cat_1', 'cat_2', 'cat_3', 'city', 'street', 'name_1',
       'name_2', 'surname', 'female', 'purchase_time', 'delivery_time',
       'delivery_company', 'delivery_duration', 'duration',
       'syntetic_duration', 'agg_time_spent', 'weekday', 'hour', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos', 'ends_with_purchase'],
      dtype='object')

### Remove unknown users

In [5]:
data.made_purchase.value_counts()

False    27839
True      3184
Name: made_purchase, dtype: int64

In [6]:
data[data.user_id.isna()].made_purchase.value_counts()

False    1377
True      152
Name: made_purchase, dtype: int64

In [7]:
clean = clean[~clean.user_id.isna()]

# Group events in sessions

In [8]:
session_grouped = clean.groupby('session_id')
session_grouped.head()

,session_id,time,user_id,product_id,event,discount,purchase_id,made_purchase,known_product,product,...,duration,syntetic_duration,agg_time_spent,weekday,hour,weekday_sin,weekday_cos,hour_sin,hour_cos,ends_with_purchase
0,100001.0,2020-01-17 16:08:57,102.0,1001.0,view_product,0,NaN,False,True,telefon siemens gigaset da310,...,00:02:43.251191,True,00:08:06.502383,4,16,-0.433884,-0.900969,-0.866025,-0.500000,False
1,100002.0,2020-01-07 05:10:41,102.0,1277.0,view_product,20,NaN,False,True,apple ipad mini 64gb 4g,...,00:03:45,False,00:05:23.251191,1,5,0.781831,0.623490,0.965926,0.258819,False
2,100002.0,2020-01-07 05:14:26,102.0,1276.0,view_product,20,NaN,False,True,apple ipad mini 64gb,...,00:02:43.251191,True,00:03:45,1,5,0.781831,0.623490,0.965926,0.258819,False
3,100003.0,2020-01-16 09:37:42,102.0,1276.0,view_product,0,NaN,False,True,apple ipad mini 64gb,...,00:00:40,False,00:06:28.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False
4,100003.0,2020-01-16 09:38:22,102.0,1277.0,view_product,0,NaN,False,True,apple ipad mini 64gb 4g,...,00:02:43.251191,True,00:09:08.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31018,110074.0,2020-02-03 05:02:30,301.0,1050.0,view_product,0,NaN,False,True,bioshock 2 (pc),...,00:02:15,False,00:05:40,0,5,0.000000,1.000000,0.965926,0.258819,False
31019,110074.0,2020-02-03 05:04:46,301.0,1051.0,view_product,0,NaN,False,True,spec ops the line (pc),...,00:02:43.251191,True,00:00:00,0,5,0.000000,1.000000,0.965926,0.258819,False
31020,110075.0,2020-03-18 07:05:52,301.0,1278.0,view_product,5,NaN,False,True,intenso music walker 8gb,...,00:02:43.251191,True,00:06:18.251191,2,7,0.974928,-0.222521,0.965926,-0.258819,False
31021,110076.0,2020-02-01 04:19:13,301.0,1001.0,view_product,10,NaN,False,True,telefon siemens gigaset da310,...,00:01:45,False,00:00:00,5,4,-0.974928,-0.222521,0.866025,0.500000,True


In [9]:
features = pd.DataFrame(index=clean.session_id.drop_duplicates())

In [10]:
products_bought = session_grouped['made_purchase'].apply(list).apply(sum)
features['products_bought'] = products_bought
products_bought.head()

session_id
100001.0    0
100002.0    0
100003.0    0
100004.0    0
100005.0    1
Name: made_purchase, dtype: int64

In [11]:
made_purchase = session_grouped['made_purchase'].any()
features['made_purchase'] = made_purchase
made_purchase.head()

session_id
100001.0    False
100002.0    False
100003.0    False
100004.0    False
100005.0     True
Name: made_purchase, dtype: bool

In [12]:
events_count = session_grouped['event'].count()
features['events_count'] = events_count
events_count.head()

session_id
100001.0    1
100002.0    2
100003.0    2
100004.0    7
100005.0    3
Name: event, dtype: int64

In [13]:
product_views = session_grouped['made_purchase'].apply(list).apply(lambda l: len([x for x in l if not x]))
features['product_views'] = product_views
product_views.head()

session_id
100001.0    1
100002.0    2
100003.0    2
100004.0    7
100005.0    2
Name: made_purchase, dtype: int64

In [14]:
total_duration = session_grouped['agg_time_spent'].max().apply(lambda x: x.total_seconds())
features['total_duration'] = total_duration
total_duration.head()

session_id
100001.0    486.502384
100002.0    323.251192
100003.0    548.251192
100004.0    230.000000
100005.0    163.251192
Name: agg_time_spent, dtype: float64

In [15]:
mean_event_duration = session_grouped['duration'].apply(np.mean).apply(lambda x: x.total_seconds())
features['mean_event_duration'] = mean_event_duration
mean_event_duration.head()

session_id
100001.0    163.251192
100002.0    194.125596
100003.0    101.625596
100004.0    186.178742
100005.0     79.417064
Name: duration, dtype: float64

In [16]:
products_viewed = session_grouped['product_id'].count()
features['products_viewed'] = products_viewed
products_viewed.head()

session_id
100001.0    1
100002.0    2
100003.0    2
100004.0    7
100005.0    3
Name: product_id, dtype: int64

In [17]:
mean_time_per_product = clean.groupby(['session_id', 'product_id'])['duration'] \
    .sum().apply(lambda x: x.total_seconds()) \
    .reset_index().groupby('session_id')['duration'].mean()
features['mean_time_per_product'] = mean_time_per_product
mean_time_per_product.head()

session_id
100001.0    163.251192
100002.0    194.125596
100003.0    101.625596
100004.0    186.178742
100005.0    119.125596
Name: duration, dtype: float64

In [18]:
user_id = session_grouped['user_id'].first()
features['user_id'] = user_id
user_id.head()

session_id
100001.0    102.0
100002.0    102.0
100003.0    102.0
100004.0    102.0
100005.0    102.0
Name: user_id, dtype: float64

In [19]:
user_sex = session_grouped['female'].first()
features['user_sex'] = user_sex
user_sex.head()

session_id
100001.0    False
100002.0    False
100003.0    False
100004.0    False
100005.0    False
Name: female, dtype: bool

In [20]:
products_viewed = session_grouped['product_id'].apply(set)
features['products_viewed'] = products_viewed
products_viewed.head()

session_id
100001.0                                             {1001.0}
100002.0                                     {1276.0, 1277.0}
100003.0                                     {1276.0, 1277.0}
100004.0    {1030.0, 1032.0, 1033.0, 1034.0, 1036.0, 1037....
100005.0                                     {1233.0, 1234.0}
Name: product_id, dtype: object

In [21]:
weekday = session_grouped[['weekday_cos', 'weekday_sin']].agg(['min', 'max'])
weekday.columns = ['_'.join(col).strip() for col in weekday.columns.values]
features = features.merge(weekday, on='session_id', suffixes=(False, False))
weekday.head()

,weekday_cos_min,weekday_cos_max,weekday_sin_min,weekday_sin_max
session_id,,,,
100001.0,-0.900969,-0.900969,-0.433884,-0.433884
100002.0,0.623490,0.623490,0.781831,0.781831
100003.0,-0.900969,-0.900969,0.433884,0.433884
100004.0,0.623490,0.623490,-0.781831,-0.781831
100005.0,1.000000,1.000000,0.000000,0.000000


In [22]:
hour = session_grouped[['hour_sin', 'hour_cos']].agg(['min', 'max'])
hour.columns = ['_'.join(col).strip() for col in hour.columns.values]
features = features.merge(hour, on='session_id', suffixes=(False, False))
hour.head()

,hour_sin_min,hour_sin_max,hour_cos_min,hour_cos_max
session_id,,,,
100001.0,-0.866025,-0.866025,-0.500000,-0.500000
100002.0,0.965926,0.965926,0.258819,0.258819
100003.0,0.707107,0.707107,-0.707107,-0.707107
100004.0,0.500000,0.707107,-0.866025,-0.707107
100005.0,0.000000,0.000000,1.000000,1.000000


In [23]:
for i in range(4):
    features = features.merge(session_grouped[f'cat_{i}'] \
        .apply(lambda c: set(pd.Series(c).dropna().unique().tolist())), on='session_id', suffixes=(False, False))

In [24]:
syntetic_duration = session_grouped['syntetic_duration'].any()
features['syntetic_duration'] = syntetic_duration
syntetic_duration.head()

session_id
100001.0    True
100002.0    True
100003.0    True
100004.0    True
100005.0    True
Name: syntetic_duration, dtype: bool

In [25]:
delivery_duration = session_grouped['delivery_duration'].apply(np.mean).apply(lambda x: x.total_seconds())
features['delivery_duration'] = delivery_duration
delivery_duration.head()

session_id
100001.0         NaN
100002.0         NaN
100003.0         NaN
100004.0         NaN
100005.0    345763.0
Name: delivery_duration, dtype: float64

In [26]:
features.head()

,products_bought,made_purchase,events_count,product_views,total_duration,mean_event_duration,products_viewed,mean_time_per_product,user_id,user_sex,...,hour_sin_min,hour_sin_max,hour_cos_min,hour_cos_max,cat_0,cat_1,cat_2,cat_3,syntetic_duration,delivery_duration
session_id,,,,,,,,,,,,,,,,,,,,,
100001.0,0,False,1,1,486.502384,163.251192,{1001.0},163.251192,102.0,False,...,-0.866025,-0.866025,-5.000000e-01,-5.000000e-01,{telefony i akcesoria},{telefony stacjonarne},{},{},True,NaN
100002.0,0,False,2,2,323.251192,194.125596,"{1276.0, 1277.0}",194.125596,102.0,False,...,0.965926,0.965926,2.588190e-01,2.588190e-01,{komputery},{tablety i akcesoria},{tablety},{},True,NaN
100003.0,0,False,2,2,548.251192,101.625596,"{1276.0, 1277.0}",101.625596,102.0,False,...,0.707107,0.707107,-7.071068e-01,-7.071068e-01,{komputery},{tablety i akcesoria},{tablety},{},True,NaN
100004.0,0,False,7,7,230.000000,186.178742,"{1030.0, 1032.0, 1033.0, 1034.0, 1036.0, 1037....",186.178742,102.0,False,...,0.500000,0.707107,-8.660254e-01,-7.071068e-01,{komputery},{monitory},{monitory lcd},{},True,NaN
100005.0,1,True,3,2,163.251192,79.417064,"{1233.0, 1234.0}",119.125596,102.0,False,...,0.000000,0.000000,1.000000e+00,1.000000e+00,{sprzęt rtv},{video},{odtwarzacze dvd},{},True,345763.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110072.0,1,True,2,1,75.000000,119.125596,{1283.0},238.251192,301.0,False,...,-0.707107,-0.707107,-7.071068e-01,-7.071068e-01,{sprzęt rtv},{video},{telewizory i akcesoria},{okulary 3d},True,NaN
110073.0,0,False,2,2,438.251192,184.125596,"{1316.0, 1319.0}",184.125596,301.0,False,...,-1.000000,-1.000000,-1.836970e-16,-1.836970e-16,{telefony i akcesoria},{akcesoria telefoniczne},{zestawy słuchawkowe},{},True,NaN
110074.0,0,False,3,3,340.000000,182.750397,"{1057.0, 1050.0, 1051.0}",182.750397,301.0,False,...,0.866025,0.965926,2.588190e-01,5.000000e-01,{gry i konsole},{gry komputerowe},{},{},True,NaN


In [27]:
features.to_pickle('data/features.pickle')